In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
#Define constants
m = 39.95*1.68*10**(-27)
epsilon = 119.8/1.381*10**(-23)
sigma = 3.405 #angstrom
h = 10**-6 #Timestep

In [3]:
#Potential function

def r(x,y):
    r_pos = np.sqrt(x**2+y**2)
    return r_pos

def Grad_potential(position_1 , position_2 , epsilon = 119.8/1.381*10**(-23), sigma = 3.405):
    #Implement image convention 
    
    direction = np.array([(position_1[0] - position_2[0]), (position_1[1] - position_2[1])])
    r = np.sqrt((position_1[0] - position_2[0])**2+(position_1[1] - position_2[1])**2)
    U_partial_r = 4 *epsilon* (12*((sigma)**(12))*(-1/r)**13 - 6*((sigma)**(6))*(-1/r)**7)
    grad_U = U_partial_r * direction *r**(-1)
    return grad_U

In [4]:
def boxcheck (x, y):
    x_upper_limit = 5.0
    y_upper_limit = 5.0
    
    x_lower_limit = 0
    y_lower_limit = 0
    
    l = x_upper_limit - x_lower_limit #length of the box, only works for squares
    
    if x < x_upper_limit and x > x_lower_limit:
        if y < y_upper_limit and y > y_lower_limit:
            return np.array([x, y])
        if y > y_upper_limit:
            return np.array([x, y -l])
        if y < y_lower_limit:
            return np.array([x, y +l])
    if x > x_upper_limit:
        if ( y < y_upper_limit and y > y_lower_limit):
            return np.array([x- l, y])
        if y > y_upper_limit:
            return np.array([x- l, y - l])
        if y < y_lower_limit:
            return np.array([x- l, y + l])
    if x < x_lower_limit:
        if ( y < y_upper_limit and y > y_lower_limit):
            return np.array([x + l, y])
        if y > y_upper_limit:
            return np.array([x + l, y - l])
        if y < y_lower_limit:
            return np.array([x + l, y + l])

        
        
#Class for argon particles
        
class Particle:
    
    mass = 39.95*1.68*10**(-27)
    def __init__(self, position, velocity):
        self.pos = position
        self.vel = velocity

In [5]:
#Put initial values in 2D arrays
#Random particle initial conditions

x_1 = random.uniform(0, 5.0)
y_1 = random.uniform(0, 5.0)
x_2 = random.uniform(0, 5.0)
y_2 = random.uniform(0, 5.0)

vx_1 = random.uniform(-50.0, 50.0)
vy_1 = random.uniform(-50.0, 50.0)
vx_2 = random.uniform(-50.0, 50.0)
vy_2 = random.uniform(-50.0, 50.0)

part_1  = np.array([[x_1],[y_1],[vx_1],[vy_1]]) #starting position and velocity for particle 1
part_2  = np.array([[x_2],[y_2],[vx_2],[vy_2]]) #starting position and velocity for particle 2

In [6]:
# Loop to initialize the particles, it still needs implementation

particles = []
n_particles = 5

for i in range(n_particles):
    
    x = random.uniform(0, 10.0)
    y = random.uniform(0, 10.0)
    position = np.array([[x],[y]])

    vx = random.uniform(-5.0, 5.0)
    vy = random.uniform(-5.0, 5.0)
    velocity = np.array([[vx],[vy]])
    particle = Particle(position , velocity)
    particles.append(particle)



In [7]:
for j in range(n_particles):
    total_acceleration_x = 0
    total_acceleration_y = 0
    for k in range(n_particles):
        if k != j:
            acceleration = Grad_potential(particles[j].pos, particles[k].pos)/m
            total_acceleration_x += acceleration[0]
            total_acceleration_y += acceleration[1]
    print(type(total_acceleration_x))
    particles[j].vel[0].append(particles[j].vel[0] - total_acceleration_x*h)
    particles[j].vel[1].append(particles[j].vel[1] - total_acceleration_y*h)
    
    particles[j].pos[0].append(particles[j].pos[0] + particles[j].vel[0, -1]*h)
    particles[j].pos[1].append(particles[j].pos[1] + particles[j].vel[1, -1]*h)

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [ ]:


#Now loop 100 times over the functions to get the new positions and velocities
timesteps = 100

for i in range(timesteps):
    #The constants don't have to be declared again in the Grad_potential function
    vx_1new = part_1[2,-1] - (Grad_potential(part_1[0,-1], part_1[1,-1], part_2[0,-1], part_2[1,-1])[0])*h/m
    vy_1new = part_1[3,-1] - (Grad_potential(part_1[0,-1], part_1[1,-1], part_2[0,-1], part_2[1,-1])[1])*h/m
    vx_2new = part_2[2,-1] - (Grad_potential(part_2[0,-1], part_2[1,-1], part_1[0,-1], part_1[1,-1])[0])*h/m
    vy_2new = part_2[3,-1] - (Grad_potential(part_2[0,-1], part_2[1,-1], part_1[0,-1], part_1[1,-1])[1])*h/m
    x_1new = part_1[0,-1] + part_1[2,-1]*h
    y_1new = part_1[1,-1] + part_1[3,-1]*h
    x_2new = part_2[0,-1] + part_2[2,-1]*h
    y_2new = part_2[1,-1] + part_2[3,-1]*h

    #Check if the particle is inside the box
    position_check_part1 = boxcheck(x_1new, y_1new)
    position_check_part2 = boxcheck(x_2new, y_2new)
    
    #add the new values for the x,y positions and velocities to the arrays for particles 1 and 2
    part_1_new = np.array([[position_check_part1[0]], [position_check_part1[1]], [vx_1new],[vy_1new]])
    part_1 = np.append(part_1, part_1_new, axis = 1)
    
    part_2_new = np.array([[position_check_part2[0]],[position_check_part2[1]], [vx_2new],[vy_2new]])
    part_2 = np.append(part_2, part_2_new, axis = 1)
    
print(part_1[0])
print(part_2[0])

In [ ]:
#Scatter plot
plt.scatter(part_1[0], part_1[1] )
plt.scatter(part_2[0], part_2[1])
plt.title("Positions of particles")
plt.xlabel("x position")
plt.ylabel("y position")
#plt.xlim(0.0, 5.)
#plt.ylim(0, 5)
plt.show()

In [ ]:
print(particles[0].pos[0])